### Embedding states |1>, |2>, |3>, |4>, & |5>

In [1]:
import numpy as np

# used for the basis states
def nestedKronecker(args): # use "*args" to access an array of inputs
    assert len(args) >= 2
    temp = args[0]
    for arg in args[1:]:
        temp = np.kron(temp, arg)
    return temp

basis = {0: [1,0], 1: [0,1], '0': [1,0], '1': [0,1]}

basisVector = lambda binstr : nestedKronecker([basis[x] for x in binstr])

# common states
zero, one = basis['0'], basis['1']
tplus = basisVector('11')
tminus = basisVector('00')
tzero = (1/np.sqrt(2))*(basisVector('01') + basisVector('10'))
singlet = np.sqrt(1/2)*(basisVector('01') - basisVector('10'))


# ------------------------ FOR STATE 1 ------------------------

state1 = np.kron(np.kron(singlet, singlet), singlet)

# ------------------------ FOR STATE 2 ------------------------

largePyramid = np.sqrt(1/3)*(np.kron(tplus,tminus)+np.kron(tminus,tplus)-np.kron(tzero,tzero))
state2 = np.kron(singlet,largePyramid)

# ------------------------ FOR STATE 3 ------------------------

state3 = np.kron(largePyramid,singlet)

# ------------------------ FOR STATE 4 ------------------------

# for psi0 and psi1 we are combining j1=1 and j2=1/2 (this is combinind the first peak and trough)
# J = 1/2, M = -1/2
psi0 = np.sqrt(1/3)*np.kron(tzero, zero) - np.sqrt(2/3)*np.kron(tminus, one)
# J = 1/2, M = +1/2
psi1 = np.sqrt(2/3)*np.kron(tplus, zero) - np.sqrt(1/3)*np.kron(tzero, one)


# for phiminus, phizero, phiplus, we are are combining j1=1/2 and j2=1/2
# J = 1, M = -1
phiminus = np.kron(psi0,zero)
# J = 1, M = 0
phizero = np.sqrt(1/2)*(np.kron(psi1,zero) + np.kron(psi0,one))
# J = 1, M = +1
phiplus = np.kron(psi1,one)

# J=0,M=0 and j1=1,j2=1
state4 = np.sqrt(1/3)*(np.kron(phiplus, tminus) - np.kron(phizero, tzero) + np.kron(phiminus, tplus))


# ------------------------ FOR STATE 5 ------------------------

eta_minus3 = np.kron(tminus, basis['0'])
eta_minus1 = np.sqrt(2/3)*np.kron(tzero,zero) + np.sqrt(1/3)*np.kron(tminus,one)
eta_plus1 = np.sqrt(1/3)*np.kron(tplus,zero) + np.sqrt(2/3)*np.kron(tzero, one)
eta_plus3 = np.kron(tplus,one)

gamma_minus = np.sqrt(1/4)*np.kron(eta_minus1, zero) - np.sqrt(3/4)*np.kron(eta_minus3, one)
gamma_zero = np.sqrt(1/2)*np.kron(eta_plus1, zero) - np.sqrt(1/2)*np.kron(eta_minus1,one)
gamma_plus = np.sqrt(3/4)*np.kron(eta_plus3, zero) - np.sqrt(1/4)*np.kron(eta_plus1, one)

state5 = np.sqrt(1/3)*(np.kron(gamma_plus,tminus) - np.kron(gamma_zero, tzero) - np.kron(gamma_minus, tplus))


In [2]:
states = [state1, state2, state3, state4, state5]


print('ensuring states are normalized (aka <s|s> ~ 1)')
for s in states:
    print(np.dot(s,s))

print('-----------------------')
total = 0
for i in range(len(states)-1):
    for j in range(len(states)-1):
        if i != j:
            temp = np.dot(states[i],states[j])
            total += temp

orthoStatement = 'First 4 states are orthogonal with each other' if total == 0 else 'First 4 states are - NOT - orthogonal with each other'
print(orthoStatement)

print('-----------------------')
i = len(states)-1
for j in range(len(states)):
    if i != j:
        temp = np.dot(states[i],states[j])
        print(f'<state5|state{j+1}> = {temp}')
        total += temp

print('Since dot product of states are close enough to zero, we conclude all states are orthogonal to each other')

ensuring states are normalized (aka <s|s> ~ 1)
1.0000000000000004
0.9999999999999999
0.9999999999999999
0.9999999999999998
0.9999999999999998
-----------------------
First 4 states are orthogonal with each other
-----------------------
<state5|state1> = 0.0
<state5|state2> = -7.554110863947324e-18
<state5|state3> = 0.0
<state5|state4> = 2.7755575615628914e-17
Since dot product of states are close enough to zero, we conclude all states are orthogonal to each other


### So above ^ I was able to prove that those states are correct. 
### --------------------------------------------------------
## Now below we are trying to use the fixed ansatz or circuit to see if we are encoding the circuit correctly
### Please edit the parts which say `TODO` and where you deem necessary

In [7]:
from scipy.linalg import expm
X = [[0,1],[1,0]]
Y = np.array([[0,-1j],[1j,0]], dtype=np.complex128)
Z = [[1,0],[0,-1]]
I = np.eye(2)
Udot = lambda s1, U, s2 : np.dot(np.conjugate(np.transpose(s1)),np.matmul(U,s2))

H_ex = (1/4)*(np.kron(X,X) + np.kron(Y,Y) + np.kron(Z,Z))

U_ex = lambda p : expm(-1j*np.pi*p*H_ex)

amp_sqrd = lambda c : np.real(c*np.conjugate(c))

# evaluation function (minimization function)
def f_CNOT(U):
    c1 = Udot(state1,U,state1)
    c2 = Udot(state2,U,state2)
    c3 = Udot(state4,U,state3)
    c4 = Udot(state3,U,state4)
    return np.sqrt(1-(1/4)*abs(amp_sqrd(c1) + amp_sqrd(c2) + amp_sqrd(c3) + amp_sqrd(c3) + amp_sqrd(c4)))


#### We need to do it in pennylane to make it easy to train ....
import pennylane as qml
from pennylane import numpy as np

np.random.seed(42)

def square_loss(targets, predictions):
    loss = 0
    for t, p in zip(targets, predictions):
        loss += (t - p) ** 2
    loss = loss / len(targets)
    return 0.5*loss

dev = qml.device('default.qubit', wires=6)

p1 = np.arccos(-1/np.sqrt(3))/np.pi
p2 = np.arcsin(1/3)/np.pi


def fixedCNOTCircuit():
    # We know this CNOT works
    # Figure 1 from "Universal Quantum Computation and Leakage Reduction in the 3-Qubit Decoherence Free Subsystem"
    qml.QubitUnitary(U_ex(p1), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(p2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(1), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(-1/2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(1), wires=[1,2])
    qml.QubitUnitary(U_ex(-1/2), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(1), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[1,2])
    qml.QubitUnitary(U_ex(1/2), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(1), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(1), wires=[1,2])
    qml.QubitUnitary(U_ex(-1/2), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(-1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(-1/2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(1), wires=[3,4])
    #
    qml.QubitUnitary(U_ex(1/2), wires=[2,3])
    qml.QubitUnitary(U_ex(1-p2), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(-p1), wires=[3,4])


def Id_n(n):
    assert n >= 0
    if n==0:
        return 1
    temp = I
    for i in range(n-1):
        temp = np.kron(temp, I)
    return temp


bounds = [[3,4],[2,5],[3,4],[2,5],[1,4],[2,5],[1,4],[2,5],[1,4],[2,5],[3,4],[2,5],[3,4]]
operators = [qml.QubitUnitary(U_ex(p1), wires=[3,4]),
qml.QubitUnitary(U_ex(1/2), wires=[2,3])@qml.QubitUnitary(U_ex(p2), wires=[4,5]),
qml.QubitUnitary(U_ex(1), wires=[3,4]),
qml.QubitUnitary(U_ex(-1/2), wires=[2,3])@qml.QubitUnitary(U_ex(-1/2), wires=[4,5]),
qml.QubitUnitary(U_ex(1), wires=[1,2])@qml.QubitUnitary(U_ex(-1/2), wires=[3,4]),
qml.QubitUnitary(U_ex(-1/2), wires=[2,3])@qml.QubitUnitary(U_ex(1), wires=[4,5]),
qml.QubitUnitary(U_ex(-1/2), wires=[1,2])@qml.QubitUnitary(U_ex(1/2), wires=[3,4]),
qml.QubitUnitary(U_ex(-1/2), wires=[2,3])@qml.QubitUnitary(U_ex(1), wires=[4,5]),
qml.QubitUnitary(U_ex(1), wires=[1,2])@qml.QubitUnitary(U_ex(-1/2), wires=[3,4]),
qml.QubitUnitary(U_ex(-1/2), wires=[2,3])@qml.QubitUnitary(U_ex(-1/2), wires=[4,5]),
qml.QubitUnitary(U_ex(1), wires=[3,4]),
qml.QubitUnitary(U_ex(1/2), wires=[2,3])@qml.QubitUnitary(U_ex(1-p2), wires=[4,5]),
qml.QubitUnitary(U_ex(-p1), wires=[3,4])]


newOps = []
for i, (start, end) in enumerate(bounds):
    temp = np.kron(np.kron(Id_n(start),qml.matrix(operators[i])), Id_n(5-end))
    newOps.append(temp.copy())

totalOperator = np.eye(2**6)
for op in newOps:
    totalOperator = np.matmul(op,totalOperator)





@qml.qnode(dev)
def quantum_model(state_):
    qml.AmplitudeEmbedding(state_, wires=range(6))
    fixedCNOTCircuit()
    return qml.state()

def target_function(U_prime):
    # approximated function
    return f_CNOT(U_prime)


expectedStates = [state1, state2, state4, state3, state5]
expectedNames = ['state1', 'state2', 'state4', 'state3', 'state5']
for i, state in enumerate(states): ############################# TODO: I think this needs to be fixed because it's outputting the wrong shit <<<<<<<<<<<<<<<<<<<-------------------------
    newstate = quantum_model(state)
    dot_val = np.dot(np.conjugate(np.transpose(newstate)), expectedStates[i])
    dot_sqrd = dot_val*np.conjugate(dot_val)
    c = amp_sqrd(Udot(expectedStates[i],totalOperator,state))
    print(f'for state{i+1}, |<{expectedNames[i]}|U_cnot|state{i+1}>|^2 : through Pennylane = {np.real(dot_sqrd)} OR matrix math = {c} \t\t(should be = 1)')


solved_U_cnot_matrix = qml.matrix(fixedCNOTCircuit)()
print(f'target_function(qml.matrix) = {target_function(np.kron(I,solved_U_cnot_matrix))}')
print(f'target_function(manual_operator) = {target_function(totalOperator)}')

for state1, |<state1|U_cnot|state1>|^2 : through Pennylane = 0.8091345264191643 OR matrix math = 0.8091345264191645 		(should be = 1)
for state2, |<state2|U_cnot|state2>|^2 : through Pennylane = 0.1596154735808354 OR matrix math = 0.1596154735808356 		(should be = 1)
for state3, |<state4|U_cnot|state3>|^2 : through Pennylane = 0.015624999999999924 OR matrix math = 0.015624999999999906 		(should be = 1)
for state4, |<state3|U_cnot|state4>|^2 : through Pennylane = 0.015624999999999983 OR matrix math = 0.01562499999999998 		(should be = 1)
for state5, |<state5|U_cnot|state5>|^2 : through Pennylane = 0.12896484472477313 OR matrix math = 0.12896484472477313 		(should be = 1)
target_function(qml.matrix) = 0.8704680005552682
target_function(manual_operator) = 0.8637671850678283


In [24]:
dev2 = qml.device('default.qubit', wires=6)

def constUnitary(const, val):
    qml.QubitUnitary(U_ex(val), wires=[0,1])
    qml.QubitUnitary(U_ex(const), wires=[2,3])
    qml.QubitUnitary(U_ex(const), wires=[4,5])
    #
    qml.QubitUnitary(U_ex(const), wires=[1,2])
    qml.QubitUnitary(U_ex(const), wires=[3,4])


originalState = basisVector('100000')

@qml.qnode(dev2)
def fixed_model(state_, val, val2):
    qml.AmplitudeEmbedding(state_, wires=range(6))
    constUnitary(val, val2)
    return qml.state()

def target_function(U_prime):
    # approximated function
    return f_CNOT(U_prime)


expectedStates = [originalState, basisVector('010000'), basisVector('010000')]
nameStates = ['100000', '010000', '010000']
values = [0,-1,+1]

for i in range(len(values)):
    newstate = fixed_model(originalState, 0, values[i])
    dot_val = np.dot(np.conjugate(np.transpose(newstate)), expectedStates[i])
    dot_sqrd = dot_val*np.conjugate(dot_val)
    print(f'|<{nameStates[i]}|U_fixed|100000>|^2 : through Pennylane = {np.real(dot_sqrd)}')


|<100000|U_fixed|100000>|^2 : through Pennylane = 1.0
|<010000|U_fixed|100000>|^2 : through Pennylane = 1.0
|<010000|U_fixed|100000>|^2 : through Pennylane = 1.0


In [4]:
# for state1, we get |<expected_state|U_cnot|state1>|^2 = 0.8091345264191643			(should be = 1)
# for state2, we get |<expected_state|U_cnot|state2>|^2 = 0.1596154735808354			(should be = 1)
# for state3, we get |<expected_state|U_cnot|state3>|^2 = 0.015624999999999924			(should be = 1)
# for state4, we get |<expected_state|U_cnot|state4>|^2 = 0.015624999999999983			(should be = 1)
# for state5, we get |<expected_state|U_cnot|state5>|^2 = 0.12896484472477313			(should be = 1)